<a href="https://colab.research.google.com/github/ZW-Shi/-/blob/master/costco_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [50]:
import os
import json
import urllib.request as urllib2
#import urllib2
import zipfile
from pprint import pprint

import numpy as np
import tensorflow as tf
import keras as k

# Load data

In [51]:
data_folder = 'data'

if not os.path.exists(data_folder):
  url = 'https://github.com/USC-Melady/KDD19-CoSTCo/releases/download/demo_data/data.zip'
  filedata = urllib2.urlopen(url)

  with open('data.zip', 'wb') as f:
      f.write(filedata.read())

  with zipfile.ZipFile('data.zip', 'r') as f:
    f.extractall('.')

In [52]:
shape = np.loadtxt(os.path.join(data_folder, 'tensor_shape.txt')).astype(int)
tr_idxs = np.loadtxt(os.path.join(data_folder, 'train_indices.txt')).astype(int)
tr_vals = np.loadtxt(os.path.join(data_folder, 'train_values.txt'))
te_idxs = np.loadtxt(os.path.join(data_folder, 'test_indices.txt')).astype(int)
te_vals = np.loadtxt(os.path.join(data_folder, 'test_values.txt'))

# Define util functions

In [53]:
def mape_keras(y_true, y_pred, threshold=0.1):
    v = k.backend.clip(k.backend.abs(y_true), threshold, None)
    diff = k.backend.abs((y_true - y_pred) / v)
    return 100.0 * k.backend.mean(diff, axis=-1)

def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return 100.0 * np.mean(diff, axis=-1).mean()

def transform(idxs):
    return [idxs[:, i] for i in range(idxs.shape[1])]

def set_session(device_count=None, seed=0):
    #gpu_options = tf.GPUOptions(allow_growth=True)
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=1)
    if device_count is not None:
        config = tf.compat.v1.ConfigProto(
            gpu_options=gpu_options,
            device_count=device_count
        )
    else:
        config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)
    sess = tf.compat.v1.Session(config=config)
    tf.compat.v1.keras.backend.set_session(sess)

    np.random.seed(seed)
    tf.compat.v1.set_random_seed(seed)
    return sess

def get_metrics(model, x, y, batch_size=1024):
    yp = model.predict(x, batch_size=batch_size, verbose=1).flatten()
    return {
        "rmse": float(rmse(y, yp)),
        "mape": float(mape(y, yp)),
        "mae": float(mae(y, yp))
    }

# Create a CoSTCo model

In [54]:
def create_costco(shape, rank, nc):
    inputs = [k.Input(shape=(1,), dtype="int32") for i in range(len(shape))]
    embeds = [
        k.layers.Embedding(output_dim=rank, input_dim=shape[i])(inputs[i])
        for i in range(len(shape))
    ]
    x = k.layers.Concatenate(axis=1)(embeds)
    x = k.layers.Reshape(target_shape=(rank, len(shape), 1))(x)
    x = k.layers.Conv2D(
        nc,
        kernel_size=(1, len(shape)),
        activation="relu",
        padding="valid"
    )(x)
    x = k.layers.Conv2D(
        nc,
        kernel_size=(rank, 1),
        activation="relu",
        padding="valid"
    )(x)
    x = k.layers.Flatten()(x)
    x = k.layers.Dense(nc, activation="relu")(x)
    outputs = k.layers.Dense(1, activation="relu")(x)
    model = k.Model(inputs=inputs, outputs=outputs)

    return model

# Set hyper-parameters

In [55]:
lr = 1e-4
rank = 20
nc = rank
epochs = 50
batch_size = 256

seed = 3
verbose = 1

# Train with early stopping

In [56]:
set_session(device_count={"GPU": 0}, seed=seed)
optim = k.optimizers.Adam(learning_rate=lr)

model = create_costco(shape, rank, nc)
model.compile(optim, loss=["mse"], metrics=["mae", mape_keras])
hists = model.fit(
    x=transform(tr_idxs),
    y=tr_vals,
    verbose=verbose,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[k.callbacks.EarlyStopping(
        monitor="val_mean_absolute_error",
        patience=10,
        restore_best_weights=True)],
);

Epoch 1/50
298/298 [==============================] - ETA: 0s - loss: 0.0299 - mae: 0.0995 - mape_keras: 57.1360

298/298 [==============================] - 22s 50ms/step - loss: 0.0299 - mae: 0.0995 - mape_keras: 57.1360 - val_loss: 0.0247 - val_mae: 0.0909 - val_mape_keras: 54.9137
Epoch 2/50
298/298 [==============================] - ETA: 0s - loss: 0.0242 - mae: 0.0896 - mape_keras: 54.2200

298/298 [==============================] - 3s 10ms/step - loss: 0.0242 - mae: 0.0896 - mape_keras: 54.2200 - val_loss: 0.0239 - val_mae: 0.0889 - val_mape_keras: 54.2757
Epoch 3/50
298/298 [==============================] - ETA: 0s - loss: 0.0228 - mae: 0.0848 - mape_keras: 51.1008

298/298 [==============================] - 3s 9ms/step - loss: 0.0228 - mae: 0.0848 - mape_keras: 51.1008 - val_loss: 0.0232 - val_mae: 0.0876 - val_mape_keras: 54.2286
Epoch 4/50
298/298 [==============================] - ETA: 0s - loss: 0.0217 - mae: 0.0825 - mape_keras: 50.1279

298/298 [==============================] - 3s 11ms/step - loss: 0.0217 - mae: 0.0825 - mape_keras: 50.1279 - val_loss: 0.0230 - val_mae: 0.0858 - val_mape_keras: 52.3884
Epoch 5/50
298/298 [==============================] - ETA: 0s - loss: 0.0211 - mae: 0.0819 - mape_keras: 50.2050

...

298/298 [==============================] - 2s 7ms/step - loss: 0.0157 - mae: 0.0687 - mape_keras: 42.2431 - val_loss: 0.0263 - val_mae: 0.0880 - val_mape_keras: 52.3571
Epoch 47/50
290/298 [============================>.] - ETA: 0s - loss: 0.0157 - mae: 0.0686 - mape_keras: 42.3134

298/298 [==============================] - 3s 9ms/step - loss: 0.0156 - mae: 0.0686 - mape_keras: 42.3933 - val_loss: 0.0267 - val_mae: 0.0889 - val_mape_keras: 52.9978
Epoch 48/50
292/298 [============================>.] - ETA: 0s - loss: 0.0155 - mae: 0.0683 - mape_keras: 42.2031

298/298 [==============================] - 2s 6ms/step - loss: 0.0155 - mae: 0.0683 - mape_keras: 42.1759 - val_loss: 0.0263 - val_mae: 0.0907 - val_mape_keras: 56.0351
Epoch 49/50
295/298 [============================>.] - ETA: 0s - loss: 0.0154 - mae: 0.0680 - mape_keras: 42.0410

298/298 [==============================] - 2s 6ms/step - loss: 0.0154 - mae: 0.0680 - mape_keras: 42.0217 - val_loss: 0.0268 - val_mae: 0.0921 - val_mape_keras: 57.4474
Epoch 50/50
293/298 [============================>.] - ETA: 0s - loss: 0.0153 - mae: 0.0680 - mape_keras: 42.1425

298/298 [==============================] - 2s 6ms/step - loss: 0.0153 - mae: 0.0680 - mape_keras: 42.1645 - val_loss: 0.0268 - val_mae: 0.0920 - val_mape_keras: 57.2375


# Evaluation model prediction over the test set

In [ ]:
tr_info = get_metrics(model, transform(tr_idxs), tr_vals)
te_info = get_metrics(model, transform(te_idxs), te_vals)

In [ ]:
pprint({'train': tr_info, 'test': te_info})